In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'..\CCM', nargout=0)
eng.addpath(r'..\CCM\util', nargout=0)

In [2]:
import sys
sys.path.append('..')

from CCM import framework as methods
from CCM import assessment as assess

import SimpleITK as sitk
import numpy as np
import pandas as pd
import os

def GetPointsFromLungCSV(filename, reference_image: sitk.Image):
    keypoints_file = pd.read_csv(filename, header=None)
    keypoints_array = np.array(keypoints_file.values.tolist())
    
    points_index = keypoints_array[:,:3]
    
    points_phy = []
    for point_index in points_index:
        points_phy.append(reference_image.TransformContinuousIndexToPhysicalPoint(point_index.tolist()))
    
    return np.array(points_phy)

path = '../Data/LUNG/'

# cases = list(range(1,4)) # for all validation cases
# cases = list(range(21,31)) # for all test cases
cases = list(range(1,2))
use_gt_points = 'landmarks'

for i in cases:
    print('Check...', i)
    case = '%04d' % i

    fixed = sitk.ReadImage(os.path.join(path, 'images', f'LungCT_{case}_0000.nii.gz'))
    moving = sitk.ReadImage(os.path.join(path, 'images', f'LungCT_{case}_0001.nii.gz'))
    
    fixed_mask = sitk.ReadImage(os.path.join(path, 'masks', f'LungCT_{case}_0000.nii.gz'))
    moving_mask = sitk.ReadImage(os.path.join(path, 'masks', f'LungCT_{case}_0001.nii.gz'))
    
    fixed_landmarks = GetPointsFromLungCSV(os.path.join(path, use_gt_points, f'LungCT_{case}_0000.csv'), fixed)
    moving_landmarks = GetPointsFromLungCSV(os.path.join(path, use_gt_points, f'LungCT_{case}_0001.csv'), moving)

    output_path = os.path.join(path, 'results', f'{case}')
    os.makedirs(output_path, exist_ok=True)

    print('\rCCM ...', end='')
    reg = methods.Registration(fixed=fixed, moving=moving, fixed_mask=fixed_mask, moving_mask=moving_mask, matlab_eng=eng, default_value=-1024)
    reg.PerformCCMRegistration(
        nlevel=[1], nmigr=[1], niter=[4000], step=1, sigma=8,
        feature_detector=reg.detectorFAST, feature_detector_size=7, feature_detector_angle=120, feature_detector_ratio=[1], feature_detector_num=[99999],
        feature_descriptor=reg.descriptorBRISK, feature_descriptor_radius=[0,3,6,9,12], feature_descriptor_samplenum=[1,100,200,400,600], feature_descriptor_dmax=7, feature_descriptor_dmin=999,
        feature_match_ratio=1, feature_match_crosscheck=True,
        route_merge=True, route_check=True, route_stay=0.001,
        ccm_k=[50], ccm_e=[0.1], ccm_j=[0.8], ccm_t=[0.5], ccm_z=[0.5])
    reg.OutputRegistrationResult(os.path.join(output_path, f'{case}_ccm_'))
    print('\rCCM, done.')
    print(assess.Assessment(registration=reg, fixed_points=fixed_landmarks, moving_points=moving_landmarks, mask=fixed_mask).AssessRegistration())

c:\Users\Administrator\Documents\Envs\crowd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Check... 1
CCM, done.
{'mse': [22080.06502600966, 31059.43134681581, 839526], 'jac': [0.8458926021337477, 0.3151326999578679, 839526], 'tre': [1.482425379953573, 1.4947699712300477, 81], 'err': [-1, -1, -1], 'SDlogJ': 2.534312135904863, 'dice': [], 'hdd': [], 'time': [136.4896879196167]}
